In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("preprocessed_data.csv")

In [ ]:
!pip install transformers tensorflow

In [ ]:
from transformers import BertTokenizer, TFBertModel

model_name = 'cahya/bert-base-indonesian-522M'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/545M [00:00<?, ?B/s]

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import time

batch_size = 32
texts = df['Tweet'].astype(str).tolist()

bert_embeddings = []

start = time.time()
for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
    batch_texts = texts[i:i+batch_size]
    encoded = tokenizer(batch_texts, return_tensors='tf', padding='max_length', truncation=True, max_length=50)
    output = model(encoded)

    bert_embeddings.append(output.last_hidden_state.numpy())
end = time.time()
print(f"Total time: {(end - start)/60:.2f} minutes")

Embedding: 100%|██████████| 412/412 [43:46<00:00,  6.38s/it]

Total time: 43.78 minutes


In [ ]:
final_embeddings = np.concatenate(bert_embeddings, axis=0)
np.save("bert_embedding.npy", final_embeddings)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/bert_embedding.npy /content/drive/MyDrive/